In [0]:
import matplotlib.pyplot as plt
import statistics
import pandas as pd
import pickle
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns

In [0]:
  from sklearn.linear_model import LinearRegression, LogisticRegression
from mord import LogisticAT

# instantiate models
model_linear = LinearRegression()
model_1vR = LogisticRegression(multi_class='ovr',
    class_weight='balanced')
model_multi = LogisticRegression(multi_class='multinomial',
    solver='lbfgs',
    class_weight='balanced')
model_ordinal = LogisticAT(alpha=0)  # alpha parameter set to zero to perform no regularisation

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with open('/content/drive/Shared drives/CS Comps: Wikipedia Article Trajectories /data/small_DF-All.pkl', 'rb') as f:
  df = pickle.load(f)

In [0]:
df.head()

,diameter,closeness,clustering,betweenness,edit count,editor count,article size,class
title,,,,,,,,
"Stockertown, Pennsylvania",19,0.145796,0.067468,0.084300,181,74,16733,B
Henry Purcell,34,0.127265,0.029097,0.009389,1489,782,32886,B
Silja Line,34,0.131410,0.121892,0.048352,387,160,29808,B
Rudolf Steiner,54,0.170678,0.192642,0.007552,3540,794,105682,B
"Richard Boyle, 1st Earl of Cork",30,0.109888,0.029974,0.047075,518,190,21479,B


In [0]:
with open('/content/drive/Shared drives/CS Comps: Wikipedia Article Trajectories /data/FA-ClassDF.pkl', 'rb') as f:
  df_fa = pickle.load(f)
with open('/content/drive/Shared drives/CS Comps: Wikipedia Article Trajectories /data/GA-ClassDF.pkl', 'rb') as f:
  df_ga = pickle.load(f)
with open('/content/drive/Shared drives/CS Comps: Wikipedia Article Trajectories /data/B-ClassDF.pkl', 'rb') as f:
  df_b = pickle.load(f)
with open('/content/drive/Shared drives/CS Comps: Wikipedia Article Trajectories /data/C-ClassDF.pkl', 'rb') as f:
  df_c = pickle.load(f)

In [0]:
for x in [df_fa, df_ga, df_b, df_c]:
  print(x.describe()[1:3])

       diameter  closeness  clustering  ...   edit count  editor count  article size
mean  38.383094   0.133415    0.064828  ...  4062.571942   1579.230216  98329.383094
std   12.057923   0.030954    0.033926  ...  3486.785157   1273.738745  46288.548945

[2 rows x 7 columns]
       diameter  closeness  clustering  ...   edit count  editor count  article size
mean  42.760761   0.116709    0.052815  ...  3122.257257   1278.454454  80724.185185
std   15.039821   0.035847    0.031692  ...  3409.710772   1238.017197  52696.317367

[2 rows x 7 columns]
       diameter  closeness  clustering  ...   edit count  editor count  article size
mean  45.397796   0.105547    0.040837  ...  2352.775551   1068.490982  63213.722445
std   15.993450   0.034887    0.026885  ...  2498.393874   1021.108936  46412.770803

[2 rows x 7 columns]
       diameter  closeness  clustering  ...   edit count  editor count  article size
mean  48.134134   0.093743    0.034688  ...  1705.273273    806.961962  46795.170170

In [0]:
df_b.describe()

,diameter,closeness,clustering,betweenness,edit count,editor count,article size
count,998.000000,998.000000,998.000000,998.000000,998.000000,998.000000,998.000000
mean,45.397796,0.105547,0.040837,0.026161,2352.775551,1068.490982,63213.722445
std,15.993450,0.034887,0.026885,0.036919,2498.393874,1021.108936,46412.770803
min,11.000000,0.028114,0.000000,0.000698,66.000000,34.000000,3205.000000
25%,34.000000,0.080283,0.022072,0.006205,738.500000,374.250000,31226.000000
50%,43.000000,0.104221,0.033960,0.013854,1546.500000,740.500000,49801.500000
75%,54.000000,0.128304,0.051465,0.029459,3013.250000,1405.250000,80339.000000
max,135.000000,0.281713,0.219537,0.449567,22387.000000,8227.000000,326539.000000


In [0]:
df_c.describe()

,diameter,closeness,clustering,betweenness,edit count,editor count,article size
count,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000
mean,48.134134,0.093743,0.034688,0.045794,1705.273273,806.961962,46795.170170
std,18.348443,0.034065,0.023260,0.059722,2015.914043,839.105654,41135.117969
min,13.000000,0.017992,0.000000,0.001026,45.000000,31.000000,2252.000000
25%,36.000000,0.068211,0.019234,0.009036,454.000000,251.000000,20186.500000
50%,45.000000,0.091165,0.029456,0.022942,983.000000,501.000000,33057.000000
75%,57.000000,0.115443,0.043816,0.055667,2104.000000,1033.500000,60748.000000
max,169.000000,0.257051,0.209212,0.429972,17464.000000,5621.000000,423356.000000


In [0]:
df.corr()[:4]

,diameter,closeness,clustering,betweenness,edit count,editor count,article size
diameter,1.000000,-0.770898,-0.521005,0.314651,-0.163877,-0.095147,-0.279421
closeness,-0.770898,1.000000,0.801852,-0.479610,0.382042,0.294377,0.481429
clustering,-0.521005,0.801852,1.000000,-0.202908,0.175285,0.024258,0.306067
betweenness,0.314651,-0.479610,-0.202908,1.000000,-0.393353,-0.429425,-0.492039


In [0]:
# Linear Regression

for dep in ("diameter", "closeness", "clustering", "betweenness"):
  y = df[dep]
  X = df[["edit count", "editor count", "article size"]]
  X = sm.add_constant(X)
  
  model = sm.OLS(y, X).fit() # sm.OLS(output, input)
  predictions = model.predict(X)
  print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               diameter   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     22.04
Date:                Tue, 03 Mar 2020   Prob (F-statistic):           2.06e-13
Time:                        20:36:54   Log-Likelihood:                -2128.4
No. Observations:                 500   AIC:                             4265.
Df Residuals:                     496   BIC:                             4282.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           49.5101      1.547     32.009   

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [0]:
# ANOVA calculation


def full_anova_table(aov):
  ''' Add more data to anova table '''
  aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
  aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
  aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
  cols = ['sum_sq', 'df', 'mean_sq', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
  aov = aov[cols]
  return aov

for dep in ("diameter", "closeness", "clustering", "betweenness"):
  print("/////////////////////////// " + dep + " ///////////////////////////")
  y = df[dep]
  article_quality = df['class']
  mod = ols(formula = "y ~ article_quality", data=df).fit()
  anova_table = sm.stats.anova_lm(mod,typ=2)
  print(full_anova_table(anova_table))


/////////////////////////// diameter ///////////////////////////
                     sum_sq     df  ...    eta_sq  omega_sq
article_quality   11743.648    4.0  ...  0.071051  0.063425
Residual         153541.190  495.0  ...       NaN       NaN

[2 rows x 7 columns]
/////////////////////////// closeness ///////////////////////////
                   sum_sq     df   mean_sq  ...        PR(>F)    eta_sq  omega_sq
article_quality  0.099827    4.0  0.024957  ...  5.288593e-14  0.128573  0.121318
Residual         0.676595  495.0  0.001367  ...           NaN       NaN       NaN

[2 rows x 7 columns]
/////////////////////////// clustering ///////////////////////////
                   sum_sq     df   mean_sq  ...        PR(>F)    eta_sq  omega_sq
article_quality  0.099492    4.0  0.024873  ...  1.098417e-17  0.158617  0.151561
Residual         0.527756  495.0  0.001066  ...           NaN       NaN       NaN

[2 rows x 7 columns]
/////////////////////////// betweenness ////////////////////////